In [1]:
import tensorflow as tf
import numpy as np
from keras import models
from keras import layers


In [2]:
from keras.datasets import boston_housing

In [3]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()


57344/57026 [==============================] - 0s 1us/step


In [9]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std


In [12]:
test_data.shape

(102, 13)

In [13]:
train_data.shape

(404, 13)

In [10]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model



In [11]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    val_data =\
        train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets =\
        train_targets[i * num_val_samples: (i+1)*num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis=0)

    partial_train_targets = np.concatenate(
        [test_targets[:i*num_val_samples], 
        train_targets[(i+1)*num_val_samples:]], axis=0)

    model = build_model()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs,
            batch_size=1, verbose=0)

    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


ValueError: Data cardinality is ambiguous:
  x sizes: 303
  y sizes: 203
Please provide data which shares the same first dimension.